# PKPD Parameter Estimation and Optimization

In [1]:
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolacors
import pickle
from tqdm import tqdm  # For progress bar
from multiprocessing import Pool, cpu_count
import os
import sys
import time
from datetime import datetime 
from datetime import timedelta
import math
import seaborn as sns

In [2]:
def run_simulation_AL_weight_adj(idx, weight, n_patients, hill_art, central_volume_exponent, pmax_artemether, ec50_artemether, pmax_lum, ec50_lum, hill_lum):
    command_weight = [
        "/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk",
        "--AL",
        "-n", str(n_patients),
        "--pmax_artemether", str(pmax_artemether),
        "--ec50_artemether", str(ec50_artemether), # ng/microliter
        "--pmax_lum",str(pmax_lum),
        "--ec50_lum", str(ec50_lum), #ng/ml
        "--hill_artemether", str(hill_art),
        "--hill_lum", str(hill_lum), 
        "--weight", str(weight),
        "--central_volume_exponent", str(central_volume_exponent),
        "-o", "1"
    ]

    try:
        # Run the command and capture output
        output_weight = subprocess.run(command_weight, capture_output=True, text=True, check=True, cwd=folder)

        # Process the output
        lines_weight = output_weight.stdout.splitlines()
        df_weight = pd.DataFrame(
            [line.split() for line in lines_weight],
            columns=["PID", "HOUR", "COMP2CONC_ARTEMETHER", "COMP2CONC_LUM", "PARASITEDENSITY"]
        )

        # Clean the DataFrame
        df_weight = df_weight.iloc[1:].apply(pd.to_numeric, errors='coerce')

        # Calculate the efficacy
        failed_treatment_count = df_weight[df_weight['PARASITEDENSITY'] >= 10].shape[0]
        total_patients = df_weight['PID'].nunique()
        successful_treatment_count = total_patients - failed_treatment_count
        efficacy = float(successful_treatment_count / total_patients) if total_patients > 0 else 0
        efficacy_percentage = (100 - ((failed_treatment_count / total_patients) * 100)) if total_patients > 0 else 0

        log_likelihood = (successful_treatment_count * np.log(efficacy + 1e-9)) + ((n_patients - successful_treatment_count) * np.log((1 - (efficacy + 1e-9))))


        return (idx, weight, hill_art, central_volume_exponent, pmax_artemether, ec50_artemether, pmax_lum, ec50_lum, hill_lum, failed_treatment_count, efficacy_percentage, log_likelihood, df_weight)

    except subprocess.CalledProcessError as e:
        print(f"Error in subprocess for weight: {weight}Kg: {e} and hill_art: {hill_art}")
        return (idx, weight, hill_art, central_volume_exponent, pmax_artemether, ec50_artemether, pmax_lum, ec50_lum, hill_lum, np.nan, None)

    except Exception as e:
        print(f"An unexpected error occurred for weight: {weight}Kg: {e} and hill_art {hill_art}")
        return (idx, weight, hill_art, central_volume_exponent, pmax_artemether, ec50_artemether, pmax_lum, ec50_lum, hill_lum, np.nan, None)

In [3]:
# Define number of workers (use all available CPU cores)
num_workers = cpu_count()
n_patients = 1000  # Number of patients for each simulation

ec50_values_artemether = [0.001] # ng/microliter, 5 ng/ml
print(f"ec50_artemether values tested: {ec50_values_artemether}")
ec50_values_lum = [0.19] # ng/microliter, so 200 ng/ml
print(f"ec50_lum values tested: {ec50_values_lum}")

pmax_values_artemether = [0.99998] #Make sure this is a list
print(f"pmax_artemether values tested: {pmax_values_artemether}")
pmax_values_lum = [0.99] #Make sure this is a list
print(f"pmax_lum values tested: {pmax_values_lum}")

test_exponent = [1] # To be used in the Hill Equation
print("Central Volume of Distribution Exponents Tested: ", test_exponent)

hill_art_range = [1]
print("Hill Coefficient for Artemether: ", hill_art_range)
hill_lum_range = [3]
print("Hill Coefficient for Lumefantrine: ", hill_lum_range)

experiment = "01"

weight_values = np.arange(5, 81, 5)  # Weights from 5kg to 80kg in steps of 5kg
print("Weight values: ", weight_values)

for exponent in test_exponent:
     file_suffix = f"MLE_MH_{experiment}"
     folder = f"{file_suffix}"
     os.makedirs(folder, exist_ok=True)  # Creates folder if it doesn't exist
     # Save a copy of the values tested
     with open(f"{folder}/{file_suffix}_parameters.txt", 'w') as file:
        file.write(f"Date and Time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n")
        file.write(f"Number of patient(s): {str(n_patients)}\n")
        file.write(f"ec50_artemether Value(s): {str(ec50_values_artemether)}\n")
        file.write(f"pmax_artemether Value(s): {str(pmax_values_artemether)}\n")
        file.write(f"Hill Co-efficient(s) Artemether: {str(hill_art_range)}\n")

        file.write(f"ec50_lum Value(s): {str(ec50_values_lum)}\n")
        file.write(f"pmax_lum Value(s): {str(pmax_values_lum)}\n")
        file.write(f"Hill Co-efficient(s) Lumefantrine: {str(hill_lum_range)}\n")
        
        file.write(f"Central Volume of Distribution Exponent(s): {str(exponent)}\n")

params_list = []
for i in range(len(weight_values)):
    for j in range(len(hill_art_range)):
        for k in range(len(test_exponent)):
            for l in range(len(pmax_values_artemether)):
                for m in range(len(ec50_values_artemether)):
                    for n in range(len(pmax_values_lum)):
                        for o in range(len(ec50_values_lum)):
                            for p in range(len(hill_lum_range)):
                                params_list.append([i, weight_values[i], n_patients, hill_art_range[j], test_exponent[k], pmax_values_artemether[l], ec50_values_artemether[m], pmax_values_lum[n], ec50_values_lum[o], hill_lum_range[p]])

print(f"\n{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}")
print(f"Parameter list: {params_list}")

ec50_artemether values tested: [0.001]
ec50_lum values tested: [0.19]
pmax_artemether values tested: [0.99998]
pmax_lum values tested: [0.99]
Central Volume of Distribution Exponents Tested:  [1]
Hill Coefficient for Artemether:  [1]
Hill Coefficient for Lumefantrine:  [3]
Weight values:  [ 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80]

2025-09-24 13:55:20
Parameter list: [[0, np.int64(5), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [1, np.int64(10), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [2, np.int64(15), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [3, np.int64(20), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [4, np.int64(25), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [5, np.int64(30), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [6, np.int64(35), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [7, np.int64(40), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [8, np.int64(45), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [9, np.int64(50), 1000, 1, 1, 0.99998, 0.001, 0.99, 0.19, 3], [10

In [ ]:
# Run simulations in parallel and track progress
efficacy_values = []
failed_treatment_count_values = []
results = []
dfs = []
weight_values_copy = []
central_volume_exponent_copy = []
pmax_values_artemether_copy = []
ec50_values_artemether_copy = []
hill_art_copy = []
pmax_values_lum_copy = []
ec50_values_lum_copy = []
hill_lum_copy = []
log_likelihood_values = []

with Pool(processes=num_workers) as pool:
    with tqdm(total=len(params_list), desc="Running simulations") as pbar:
        for result in pool.starmap(run_simulation_AL_weight_adj, params_list):
            idx, weight, hill_art, central_volume_exponent, pmax_artemether, ec50_artemether, pmax_lum, ec50_lum, hill_lum, failed_treatment_count, efficacy_percentage, log_likelihood, df_weight = result
            results.append((idx, weight, hill_art, central_volume_exponent, pmax_artemether, ec50_artemether, pmax_lum, ec50_lum, hill_lum, failed_treatment_count, log_likelihood, efficacy_percentage, df_weight))
            if df_weight is not None:
                dfs.append(df_weight)
            pbar.update(1)  # Update progress bar after each result

# Update the efficacy array with the results
for idx, weight, hill_art, central_volume_exponent, pmax_artemether, ec50_artemether, pmax_lum, ec50_lum, hill_lum, failed_treatment_count, efficacy_percentage, log_likelihood,df_weight in results:
    weight_values_copy.append(weight)
    efficacy_values.append(efficacy_percentage)
    central_volume_exponent_copy.append(central_volume_exponent)
    pmax_values_artemether_copy.append(pmax_artemether)
    ec50_values_artemether_copy.append(ec50_artemether)
    hill_art_copy.append(hill_art)
    pmax_values_lum_copy.append(pmax_lum)
    ec50_values_lum_copy.append(ec50_lum)
    hill_lum_copy.append(hill_lum)
    failed_treatment_count_values.append(failed_treatment_count)
    log_likelihood_values.append(log_likelihood)
    
df = pd.DataFrame({
    'Weight': weight_values_copy,
    'central_volume_exponent': central_volume_exponent_copy,
    'pmax_artemether': pmax_values_artemether_copy,
    'ec50_artemether': ec50_values_artemether_copy,
    'hill_artemether': hill_art_copy,
    'pmax_lum': pmax_values_lum_copy,
    'ec50_lum': ec50_values_lum_copy,
    'hill_lum': hill_lum_copy,
    'failed_treatment_count': failed_treatment_count_values,
    'Log_Likelihood': log_likelihood_values,
    'Efficacy': efficacy_values}) 

# Save the DataFrame to a pickle file

file_suffix = f"AL_model_calibration_experiment_{experiment}"

df.to_pickle(path = f"{folder}/pkpd_weight_vs_efficacy_results_{file_suffix}.pyobj" )

# Save all individual DataFrames to a single pickle file
df_all = pd.concat(dfs, ignore_index=True)
df_all.to_pickle(path = f"{folder}/pkpd_weight_vs_efficacy_results_parasite_density_{file_suffix}.pyobj")

# Save the efficacy DataFrame to a CSV file
df.to_csv(path_or_buf= f"{folder}/pkpd_weight_vs_efficacy_results_{file_suffix}.csv", index=False)

print(f"{file_suffix}: DataFrame saved as pickle and CSV files successfully.")

Running simulations:   0%|          | 0/16 [00:00<?, ?it/s]